In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import warnings
import seaborn as sns
from sklearn.ensemble  import RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [2]:
order_df=pd.read_parquet('tezcatli_orders_data.parquet')
dodge_df=pd.read_excel('Housing Starts Analytics.xlsx')
rr_data=pd.read_csv('r&r.csv')

In [3]:
order_df['Order Create Date']=pd.to_datetime(order_df['Order Create Date'])
order_df=order_df.loc[order_df['Region Name']!='Export/Interco']
# order_df=order_df.loc[order_df['Product Segment'].isin(['Exterior','Interior'])]
order_df=order_df[[ 'Order Create Date',
       'Region Name', 'District Name', 'forecast_group', 'Order Volume (STD)']]

order_df['year'] = order_df['Order Create Date'].dt.year
order_df['month'] = order_df['Order Create Date'].dt.month
order_df=order_df.sort_values(by='Order Create Date').reset_index(drop=True)
def convert_to_year_quarter(year,month):
    year = year
    quarter = (month - 1) // 3 + 1
    return f'{year}Q{quarter}'
order_df['Calendar Year']=order_df.apply(lambda x:convert_to_year_quarter(x['year'],x['month']),axis=1)
forecast_group='Pr Plk'
order_df_filter=order_df.loc[order_df['forecast_group']=='Pr Plk']
order_df_filter=order_df_filter.groupby(['Calendar Year','Region Name','District Name'])['Order Volume (STD)'].sum().reset_index()

In [82]:
# dodge_df=dodge_df.loc[dodge_df['Housing Type']!='High']

In [101]:
# dodge_df_sf=dodge_df.loc[dodge_df['Housing Type']=='Detached']
# dodge_df_mf=dodge_df.loc[dodge_df['Housing Type']=='Low']

In [103]:
# dodge_df_sf['Calendar - Quarter'].replace({'Q1':'Q2','Q2':'Q3','Q3':'Q4','Q4':'Q1'},inplace=True)
# dodge_df['Calendar - Quarter'].replace({'Q1':'Q7','Q7':'Q9'})

In [4]:
# dodge_df_sf.loc[dodge_df_sf['Calendar - Quarter']=='Q1',['Calendar - Year']]=dodge_df_sf.loc[dodge_df_sf['Calendar - Quarter']=='Q1','Calendar - Year']+1

In [5]:
# dodge_df_sf.reset_index(drop=True,inplace=True)

In [6]:
dodge_df['Calendar Year']=dodge_df['Calendar - Year'].astype('str')+dodge_df['Calendar - Quarter'].astype('str')
dodge_df=dodge_df.drop(['Calendar - Year','Calendar - Quarter'],axis=1)
dodge_df=dodge_df.groupby(['Calendar Year','Region Name'])['Units'].sum().reset_index()

In [7]:
merge_df=order_df_filter.merge(dodge_df,on=['Calendar Year','Region Name'],how='left')
# merge_df['Calendar Year']=pd.to_datetime(merge_df['Calendar Year'])

In [8]:
rr_data
rr_data=rr_data.drop(['District','District Number','Sub Division','Division'],axis=1)


In [9]:
merge_df=merge_df.merge(rr_data,on=['Calendar Year','Region Name','District Name'],how='left')

In [10]:
merge_df.duplicated().sum()

0

In [11]:
df=merge_df.loc[merge_df['Region Name']=='Carolinas']

In [12]:
df['Calendar Year']=pd.to_datetime(df['Calendar Year'])
df.reset_index(drop=True,inplace=True)

In [13]:
df=df.groupby(by=['Calendar Year'])['Project Count','Units','Order Volume (STD)'].sum().reset_index()

In [14]:
df=df.set_index('Calendar Year')

In [15]:
df

,Project Count,Units,Order Volume (STD)
Calendar Year,,,
2013-04-01,98436.0,74130.000000,7271118.25
2013-07-01,99557.0,70728.000000,7009235.21
2013-10-01,100268.0,64470.000000,6759348.91
2014-01-01,100760.0,60822.000000,6466595.75
2014-04-01,101248.0,71106.000000,7426870.50
2014-07-01,101924.0,63975.000000,7962189.75
2014-10-01,102639.0,64359.000000,7029540.50
2015-01-01,103859.0,70485.000000,7168168.00
2015-04-01,105031.0,84378.000000,6580265.75


In [57]:
seq_size=5
x=[]
y=[]

for i in range(len(df) - seq_size - 1):
    # Extract the current window (input sequence) from the DataFrame
    w=df.iloc[i:(i + seq_size), :]
    x.append(list(np.array(w).ravel()))
    w=df.iloc[(i + seq_size), :]
    y.append(list(np.array(w).ravel()))
    
    

In [ ]:
def to_sequences(df, seq_size=1):

    x = []  # List to store the input sequences (windows)
    y = []  # List to store the output values (targets)

    for i in range(len(df) - seq_size - 1):
        # Extract the current window (input sequence) from the DataFrame
        window = df[i:(i + seq_size), 0]
        x.append(list(np.array(window).ravel()))

        # Get the target value corresponding to the next time step after the input sequence
        y.append(df[i + seq_size, 0])

    # Convert the lists to numpy arrays for further processing
    return np.array(x), np.array(y)


In [58]:
model=RandomForestRegressor(n_estimators=30)

In [60]:
x=np.array(x)
y=np.array(y)

In [61]:
x=x[:-6,:]
y=y[:-6,:] 
x_test=x[-6:,:]
y_test=y[-6:,:]

In [62]:
model.fit(x,y)

RandomForestRegressor(n_estimators=30)

In [64]:

y_p=model.predict(x_test)

In [70]:
y

array([[  101924.  ,    63975.  ,  7962189.75],
       [  102639.  ,    64359.  ,  7029540.5 ],
       [  103859.  ,    70485.  ,  7168168.  ],
       [  105031.  ,    84378.  ,  6580265.75],
       [  106102.  ,    83055.  ,  7152415.75],
       [  107064.  ,    67350.  ,  6445479.5 ],
       [  107992.  ,    86922.  ,  7866818.25],
       [  108915.  ,    87495.  ,  8921523.75],
       [  109830.  ,    77007.  ,  8248350.25],
       [  110801.  ,    75951.  ,  8585117.75],
       [  111949.  ,    97170.  , 11186280.  ],
       [  113272.  ,    92103.  ,  9685697.5 ],
       [  114428.  ,    95790.  ,  8832297.  ],
       [  115160.  ,    87126.  ,  9248080.  ],
       [  118310.  ,    94845.  ,  9872629.75],
       [  119607.  ,   103293.  ,  8952403.75],
       [  120418.  ,    92559.  ,  9797059.5 ],
       [  120473.  ,    85086.  ,  9199171.5 ],
       [  123086.  ,    84900.  , 12575908.  ],
       [  124151.  ,   101208.  , 10323252.75],
       [  125480.  ,    96879.  , 101721

In [71]:
y_test

array([[  138788.  ,   116544.  , 12439620.25],
       [  141335.  ,   114132.  , 13672524.5 ],
       [  147070.  ,   123960.  , 13119076.  ],
       [  151139.  ,   133113.  , 17920039.5 ],
       [  155714.  ,   134589.  , 18501651.25],
       [  160295.  ,   110817.  , 19493217.  ]])

In [66]:
mean_absolute_error(y_test[:,2],y_p[:,2])

767177.4888888892

In [67]:
mean_absolute_percentage_error(y_test[:,2],y_p[:,2])

0.04681774322511937